In [128]:
import tensorflow as tf
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import timeit

In [129]:
filenames = []
for file in os.listdir('audio'):
    filenames.append(file)
# filenames[0:2]
print("num of files=",len(filenames),",num of actors=",len(filenames)/16)

num of files= 384 ,num of actors= 24.0


In [130]:
random.shuffle(filenames)

In [131]:
# max, min = 76117, 53310
# start, end = 21000/sampling_rate, 47424/sampling_rate
# wave_length = end-start
num_train = int(len(filenames)*0.7)
num_test = len(filenames) - num_train

In [132]:
data_x_train = []
data_x_test = []

start_time = timeit.default_timer()
for filename in filenames[:num_train]:
    data, sampling_rate = librosa.load("audio/" + filename, sr=22050*2, res_type='kaiser_fast', duration=2.5, offset=0.5)
    sampling_rate = np.array(sampling_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=13), axis=0)
    data_x_train.append(mfccs)
    
for filename in filenames[num_train:]:
    data, sampling_rate = librosa.load("audio/" + filename, sr=22050*2, res_type='kaiser_fast', duration=2.5, offset=0.5)
    sampling_rate = np.array(sampling_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=13), axis=0)
    data_x_test.append(mfccs)
    
stop_time = timeit.default_timer()
print('runtime: ', stop_time - start_time)  

runtime:  48.07236805319144


In [133]:
data_y_train = []
data_y_test = []

for filename in filenames[:num_train]:
    if int(filename[19])%2 == 0:
        data_y_train.append(1)
    else:
        data_y_train.append(0)
        
for filename in filenames[num_train:]:
    if int(filename[19])%2 == 0:
        data_y_test.append(1)
    else:
        data_y_test.append(0) 

In [134]:
# takes every value in data_y to an array
# ex: [0, 1, 1, 1, 0] => [[0], [1], [1], [1], [0]]
data_y_train_correct = []
data_y_test_correct = []

for val in data_y_train:
    val_arr = []
    val_arr.append(val)
    data_y_train_correct.append(val_arr)
    
for val in data_y_test:
    val_arr = []
    val_arr.append(val)
    data_y_test_correct.append(val_arr)

In [135]:
len(data_y_test)

116

In [136]:
data_xy_train = []
for i in range(len(data_x_train)):
    data_xy_train.append( (data_x_train[i], data_y_train_correct[i]) )
    
data_xy_test = []
for i in range(len(data_x_test)):
    data_xy_test.append( (data_x_test[i], data_y_test_correct[i]) )

In [137]:
def getXvalues(data_xy):
    x_values = []
    for data in data_xy:
        x_values.append(data[0])
    return x_values

def getYvalues(data_xy):
    y_values = []
    for data in data_xy:
        y_values.append(data[1])
    return y_values

In [138]:
def logistic_fun(z):
    return 1/(1.0 + np.exp(-z))

In [139]:
len(data_xy_train[1][0])

216

In [127]:
features = len(data_xy_train[0][0])
eps = 1e-12

x = tf.placeholder(tf.float32, [None, features])
y_ = tf.placeholder(tf.float32, [None, 1])
W = tf.Variable(tf.zeros([features,1]))
b = tf.Variable(tf.zeros([1]))
y = tf.nn.sigmoid(tf.matmul(x,W)+b)

loss1 = -(y_ * tf.log(y + eps) + (1 - y_) * tf.log(1 - y + eps))
loss = tf.reduce_mean(loss1)
update = tf.train.GradientDescentOptimizer(0.00002).minimize(loss)

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

loss_history_train = []
loss_history_test = []
accuracy_history = []

start_time = timeit.default_timer()

for i in range(0,100):
    sess.run(update, feed_dict = {x:getXvalues(data_xy_train), y_:getYvalues(data_xy_train)}) #BGD
    loss_history_train.append(np.mean(sess.run(loss1, feed_dict = {x:getXvalues(data_xy_train), y_:getYvalues(data_xy_train)})))
    random.shuffle(data_xy_train)
    
    if i%10 == 0:
        right = 0
        for i in range(len(data_x_test)):
            pred = logistic_fun(np.matmul(getXvalues(data_xy_train)[i],sess.run(W)) + sess.run(b))
            loss_history_test.append(np.mean(sess.run(loss1, feed_dict = {x:getXvalues(data_xy_test), y_:getYvalues(data_xy_test)})))
            if data_y_test[i] == 0 and pred < 0.5:
                right += 1
            if data_y_test[i] == 1 and pred > 0.5:
                right += 1
        accuracy_history.append(right/num_test)

stop_time = timeit.default_timer()
print('runtime: ', stop_time - start_time)  

NameError: name 'config' is not defined

In [186]:
features = len(data_xy_train[0][0])
hidden_layer_nodes = 28

x = tf.placeholder(tf.float32, [None, features])
y_ = tf.placeholder(tf.float32, [None, 1])
W1 = tf.Variable(tf.truncated_normal([features,hidden_layer_nodes], stddev=0.1))
b1 = tf.Variable(tf.constant(0.1, shape=[hidden_layer_nodes]))
z1 = tf.add(tf.matmul(x,W1),b1)
a1 = tf.nn.relu(z1)
W2 = tf.Variable(tf.truncated_normal([hidden_layer_nodes,1], stddev=0.1))
b2 = tf.Variable(0.)
z2 = tf.matmul(a1,W2) + b2
y = 1 / (1.0 + tf.exp(-z2))

loss = tf.reduce_mean(-(y_ * tf.log(y) + (1-y_) * tf.log(1-y)))
update = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

loss_history_train = []
loss_history_test = []
accuracy_history = []

start_time = timeit.default_timer()

for i in range(0,2000):
    sess.run(update, feed_dict = {x:getXvalues(data_xy_train), y_:getYvalues(data_xy_train)})
#     loss_history_train.append(np.mean(sess.run(loss, feed_dict = {x:getXvalues(data_xy_train), y_:getYvalues(data_xy_train)})))
    
    if False:
        pred = y.eval(session=sess, feed_dict = {x:getXvalues(data_xy_test), y_:getYvalues(data_xy_test)})
        for i in range(len(data_x_test)):
            loss_history_test.append(np.mean(sess.run(loss, feed_dict = {x:getXvalues(data_xy_test), y_:getYvalues(data_xy_test)})))
            if data_y_test[i] == 0 and pred[i][0] < 0.5:
                tn += 1
            elif data_y_test[i] == 0 and pred[i][0] > 0.5:
                fn += 1
            if data_y_test[i] == 1 and pred[i][0] > 0.5:
                tp += 1
            elif data_y_test[i] == 1 and pred[i][0] < 0.5:
                fp += 1
        accuracy_history.append((tp+tn)/(tp+tn+fp+fn))
    
stop_time = timeit.default_timer()
print('runtime: ', stop_time - start_time)  

runtime:  8.204145701522975


In [184]:
# data_x_train =          [arr1, arr2, arr3]       # data_x_test = [arr4, arr5]
# shuffle(data_x_train) = [arr2, arr3, arr1]
# data_y_train = [0, 1, 1] # data_y_test = [0, 1]

tp, tn, fp, fn = 0,0,0,0

pred = y.eval(session=sess, feed_dict = {x:getXvalues(data_xy_test), y_:getYvalues(data_xy_test)})
for i in range(len(data_x_test)):
    if data_y_test[i] == 0 and pred[i][0] < 0.5:
        tn += 1
    elif data_y_test[i] == 0 and pred[i][0] > 0.5:
        fn += 1
    if data_y_test[i] == 1 and pred[i][0] > 0.5:
        tp += 1
    elif data_y_test[i] == 1 and pred[i][0] < 0.5:
        fp += 1
#     print("pred:",pred,"=",np.round(pred),"true:",data_y_test[i])
print("tp:",tp,",tn",tn,",fp:",fp,",fn:",fn)

tp: 48 ,tn 51 ,fp: 9 ,fn: 8


In [185]:
# learning rate = 0.0001, iter = 2000
#10 nodes
# accuracy: 0.8448275862068966
# recall: 0.819672131147541
# precision: 0.8771929824561403

# 20 nodes
# accuracy: 0.8706896551724138
# recall: 0.8620689655172413
# precision: 0.8771929824561403

# 25 nodes
# accuracy: 0.8879310344827587
# recall: 0.8548387096774194
# precision: 0.9298245614035088

# 26 nodes
# accuracy: 0.853448275862069
# recall: 0.8571428571428571
# precision: 0.8421052631578947
    
print("accuracy:",(tp+tn)/(tp+tn+fp+fn))
print("recall:",(tp)/(tp+fn))
print("precision:",(tp)/(tp+fp))

accuracy: 0.853448275862069
recall: 0.8571428571428571
precision: 0.8421052631578947


In [ ]:
plt.plot(loss_history_train)

In [ ]:
plt.plot(loss_history_test)

In [ ]:
plt.plot(accuracy_history)

In [151]:
self_test = []
data, sampling_rate = librosa.load("shlomi2.wav", sr=22050*2, res_type='kaiser_fast', duration=2.5, offset=0.5)
sampling_rate = np.array(sampling_rate)
mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=13), axis=0)
self_test.append(mfccs)

In [114]:
self_test = []
data, sampling_rate = librosa.load("tali.wav", sr=22050*2, res_type='kaiser_fast', duration=2.5, offset=0.5)
sampling_rate = np.array(sampling_rate)
mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=13), axis=0)
self_test.append(mfccs)

In [152]:
# pred = logistic_fun(np.matmul(self_test,sess.run(W)) + sess.run(b))
pred = y.eval(session=sess, feed_dict = {x:self_test, y_:[[0]]})

pred

array([[0.74546957]], dtype=float32)